# Machine Learning prediction of heatwaves
**Alessandro Lovo and George Miloshevich**

*To see equations in this file you may need Markdown Math extension on your vs code for example...*

Below we will need some standard routines but we also introduce utilities.py that allows more efficient processing of dictionaries and json files for example.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import os
from pathlib import Path

import utilities as ut

# Data

## Fetch the data

The first procedure involves downloading the data to the local space which we have hosted on owncloud

In [2]:
data_info_dict = ut.json2dict('data_info.json')

def fetch(name, destination='./data', reload=False):
    destination = Path(destination).resolve()
    if not os.path.exists(destination):
        destination.mkdir(parents=True)

    if name.endswith('.nc'):
        name = name[:-3]

    file = destination / f'{name}.nc'
    if not os.path.exists(file) or reload:
        try:
            link = data_info_dict[name]['link']
        except KeyError:
            raise KeyError(f'No item {name} in data_info.json')
        os.system(f"wget -O {file} \"{link}\"")
    else:
        print('file already loaded')

## Working with climate data
We need to understand how the data looks, whether there are some outright anomalies, lest we succumb to the consequences of the principle:
> ``garbage in garbage out''
>
> Used to express the idea that in computing and other spheres, incorrect or poor quality input will always produce faulty output (often abbreviated as GIGO). The saying is recorded from the mid 20th century.

### Plasim data
* Look at some overall statistics, mean, std, etc

    - t2m: two meter temperature
    - zg500: 500 mbar geopotential
    - mrso: soil moisture
    
* plot few times with geocontour and geocontourf that we provide

### Area integrals
* Define the area of France (or some other place)
$$
\alpha(t):=\frac{1}{|\mathcal{D}|} \int_{D}\left(T_{2m}-\mathbb{E}\left(T_{2m}\right)\right)(\vec{r}, u) \mathrm{d} \vec{r} \mathrm{~d} u
$$
* Compute area intregrals
* Look at the time series, statistics, mean, std
* Compute the running mean of T-days (T=14 days)
    $$
        A(t):=\frac{1}{T} \int_{t}^{t+T} \alpha(t)  
    $$

## Define labels 
* Define heat wave labels (Choose 95 percentile of $A(t)$ for example)
---

# Machine Learning

### Probabilistic Prediction
* Familiarize yourself with skill scores
    - Brier score:
        $$ 
            S=\frac{1}{2}\sum_{k=0}^{K-1}\left[\delta_{Y,k}-\hat{p}_{k}(X)\right]^{2}
        $$
    - Logarithmic score
    - Matthew's Correlation Coefficient ($\phi$-coefficient)
* The goal is to predict probabilities

### Simple classifier
Before trying something complex always resort to good-old techniques to see what can be achieved without much effort
* Perform train-validation-test 
* Try undersampling the negative labels to accelerate learning and save some RAM
* Logistic regression, Naive Bayes, kNN, etc
* See what can be achieved with simple persistance
* Try using soil moisture or geopotential in the same spirit

# Neural networks

### Convolutional Neural Network
* Design a simple CNN architecture with a dense layer and sigmoid/softmax output
* Start by providing input with 2-meter temperature, or geopotential (results will vary)
* Normalize the original data (try different ways)
* Put the data through the pipeline and train the network

# Hyperparameter optimization

* Try stacking different fields together
* Implement early stopping to save time

You can try performing grid-search, random-search or bayesian-search using optuna (recommended)

### optimization using `optuna`

#### Example

In [2]:
import optuna

In [8]:
# define a function that takes as input a trial and returns a float

def objective(trial):
    # obtain a value for your hyperparameters using the functions `suggest_float`, `suggest_int` and `suggest_categorical`

                        # parameter name
                        #    |    bounds
    x = trial.suggest_float('x', -10, 10)
    y = trial.suggest_float('y', -10, 10)

    # compute the score related to the given hyperparameters:
    # i.e. build your network, train it and evaluate it on the validation set

    score = (x - 2)**2 + 0.1*y**2 # here we use a dummy score
    return score

study = optuna.create_study() # create a study

study.optimize(objective, n_trials=100,)


[I 2022-03-30 16:08:50,022] A new study created in memory with name: no-name-a1138ac3-cee3-4b18-aa88-f230a848f8c7
[I 2022-03-30 16:08:50,026] Trial 0 finished with value: 124.12790496006869 and parameters: {'x': -8.901355509694792, 'y': -7.272106305133123}. Best is trial 0 with value: 124.12790496006869.
[I 2022-03-30 16:08:50,027] Trial 1 finished with value: 41.03006023561601 and parameters: {'x': -4.403643315394518, 'y': 0.48386490696386275}. Best is trial 1 with value: 41.03006023561601.
[I 2022-03-30 16:08:50,029] Trial 2 finished with value: 8.593765906220533 and parameters: {'x': 4.632372041325553, 'y': -4.079685456341058}. Best is trial 2 with value: 8.593765906220533.
[I 2022-03-30 16:08:50,030] Trial 3 finished with value: 8.35635166331778 and parameters: {'x': 0.28799159429852494, 'y': 7.36571712878337}. Best is trial 3 with value: 8.35635166331778.
[I 2022-03-30 16:08:50,032] Trial 4 finished with value: 78.15319489647251 and parameters: {'x': -6.836189159983581, 'y': 0.865

In [13]:
study.best_trial.number, study.best_params

(86, {'x': 1.8716851306019044, 'y': 0.43584193845472774})

In [10]:
df = study.trials_dataframe()
df

,number,value,datetime_start,datetime_complete,duration,params_x,params_y,state
0,0,124.127905,2022-03-30 16:08:50.025571,2022-03-30 16:08:50.026106,0 days 00:00:00.000535,-8.901356,-7.272106,COMPLETE
1,1,41.030060,2022-03-30 16:08:50.027116,2022-03-30 16:08:50.027620,0 days 00:00:00.000504,-4.403643,0.483865,COMPLETE
2,2,8.593766,2022-03-30 16:08:50.028680,2022-03-30 16:08:50.029133,0 days 00:00:00.000453,4.632372,-4.079685,COMPLETE
3,3,8.356352,2022-03-30 16:08:50.030210,2022-03-30 16:08:50.030663,0 days 00:00:00.000453,0.287992,7.365717,COMPLETE
4,4,78.153195,2022-03-30 16:08:50.031673,2022-03-30 16:08:50.032093,0 days 00:00:00.000420,-6.836189,0.865771,COMPLETE
...,...,...,...,...,...,...,...,...
95,95,0.545712,2022-03-30 16:08:51.057898,2022-03-30 16:08:51.067346,0 days 00:00:00.009448,2.694544,0.795746,COMPLETE
96,96,3.465553,2022-03-30 16:08:51.068611,2022-03-30 16:08:51.077414,0 days 00:00:00.008803,3.858855,0.319570,COMPLETE
97,97,1.428378,2022-03-30 16:08:51.078392,2022-03-30 16:08:51.090125,0 days 00:00:00.011733,0.949887,1.804554,COMPLETE
98,98,0.167626,2022-03-30 16:08:51.091650,2022-03-30 16:08:51.101486,0 days 00:00:00.009836,1.949002,-1.284623,COMPLETE


#### Optuna has several nice tools for visualization of the hyperparameter optimization

In [11]:
optuna.visualization.plot_optimization_history(study)

In [4]:
fig = optuna.visualization.plot_contour(study, params=['x', 'y'])
fig.show()

In [14]:
optuna.visualization.plot_param_importances(study)